# Discard Model: NN_v1

We use tensorflow to train our NN model.

Testdata from [Japanese Mahjong Board States](https://www.kaggle.com/datasets/trongdt/japanese-mahjong-board-states)

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os

In [33]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


## Training part

In [36]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(34,)),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(34, activation='softmax')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### Prepare training data

In [37]:
# # 載入數據
# folder_path = 'dataset_extracted_2'
# all_data = pd.DataFrame()
# dataframes = []
# cnt = 0
# for file in os.listdir(folder_path):
#     if file.endswith('.csv'):
#         file_path = os.path.join(folder_path, file)
#         data = pd.read_csv(file_path)
#         dataframes.append(data)
#         cnt+=1
#         if(cnt > 2000):
#             break

# all_data = pd.concat(dataframes, ignore_index=True)
# X = all_data.iloc[:, 68:102].values
# Y = all_data.iloc[:, -1].values
# Y = tf.keras.utils.to_categorical(Y)
# np.save('dataX.npy', X)
# np.save('dataY.npy', Y)

In [38]:
X = np.load('dataX.npy')
Y = np.load('dataY.npy')
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(776752, 34)
(776752, 34)
(194188, 34)
(194188, 34)


### Checkpoint

In [39]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    filepath='checkpoints/model_epoch_{epoch:02d}.h5', 
    save_weights_only=True,
    save_freq='epoch',
    verbose=1
)


### Training

In [40]:
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_val, y_val), callbacks=[checkpoint])

Epoch 1/10
12137/12137 [==============================] - 76s 6ms/step - loss: 2.0173 - accuracy: 0.3691 - val_loss: 1.7212 - val_accuracy: 0.4294

Epoch 00001: saving model to checkpoints\model_epoch_01.h5
Epoch 2/10
12137/12137 [==============================] - 80s 7ms/step - loss: 1.7212 - accuracy: 0.4337 - val_loss: 1.6232 - val_accuracy: 0.4578

Epoch 00002: saving model to checkpoints\model_epoch_02.h5
Epoch 3/10
12137/12137 [==============================] - 79s 6ms/step - loss: 1.6486 - accuracy: 0.4527 - val_loss: 1.5709 - val_accuracy: 0.4703

Epoch 00003: saving model to checkpoints\model_epoch_03.h5
Epoch 4/10
12137/12137 [==============================] - 82s 7ms/step - loss: 1.6126 - accuracy: 0.4626 - val_loss: 1.5521 - val_accuracy: 0.4794

Epoch 00004: saving model to checkpoints\model_epoch_04.h5
Epoch 5/10
12137/12137 [==============================] - 78s 6ms/step - loss: 1.5875 - accuracy: 0.4688 - val_loss: 1.5261 - val_accuracy: 0.4834

Epoch 00005: saving mode

In [41]:
model.load_weights('checkpoints\model_epoch_10.h5')

In [42]:
def filter_prediction(input_tiles, prediction):
    legal_tiles = np.where(input_tiles > 0)[1]
    prediction = prediction.reshape(34,)
    filtered_prediction = np.zeros_like(prediction)
    filtered_prediction[legal_tiles] = prediction[legal_tiles]
    return np.argmax(filtered_prediction)

def discard(hands_input):
    hands = np.zeros(34) 

    for tile in hands_input:
        hands[tile['type'] * 9 + tile['index'] - 1] += 1

    hands = hands.reshape(1, 34)
    out = filter_prediction(hands, model.predict(hands))

    for i, tile in enumerate(hands_input):
        if tile['type'] * 9 + tile['index'] - 1 == out:
            return i

In [43]:

hands = [1, 0, 0, 0, 0, 0, 0, 0, 0,  3, 0, 0, 0, 0, 0, 0, 0, 1,  1, 0, 0, 0, 0, 0, 0, 0, 1,  1, 1, 1, 1, 1, 1, 1]
#new =   [0, 0, 0, 0, 0, 0, 0, 0, 0,  1, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0]
new = 9

hands_input = [
    {'dora': False, 'type': 0, 'index': 3},
    {'dora': False, 'type': 0, 'index': 2},
    {'dora': False, 'type': 0, 'index': 3},
    {'dora': False, 'type': 1, 'index': 4},
    {'dora': False, 'type': 1, 'index': 5},
    {'dora': False, 'type': 1, 'index': 6},
    {'dora': False, 'type': 1, 'index': 7},
    {'dora': False, 'type': 1, 'index': 8},
    {'dora': False, 'type': 1, 'index': 9},
    {'dora': False, 'type': 2, 'index': 0},
    {'dora': False, 'type': 3, 'index': 1},
    {'dora': False, 'type': 3, 'index': 2},
    {'dora': False, 'type': 3, 'index': 3},
    {'dora': False, 'type': 3, 'index': 3}
]
print(discard(hands_input))

10


## Organize into Class
merge above code into a single class, for the convenience of future use

In [ ]:
class Model_NN():
    def __init__(self) -> None:
        self.model = Sequential([Dense(64, activation='relu', input_shape=(34,)),
                                 Dense(128, activation='relu'),
                                 Dense(256, activation='relu'),
                                 Dropout(0.2),
                                 Dense(34, activation='softmax')
                                ])
        self.model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3),loss='categorical_crossentropy', metrics=['accuracy'])       
        self.model.load_weights('checkpoints\model_epoch_10.h5')

    def filter_prediction(self, input_tiles, prediction):
        """
        filter the prediction to make sure the output is a legal tile index
        
        Args:
            input_tiles (np.array): player's hand (length = 34 vec, one-hot encoding)
            prediction (np.array): model's prediction (length = 34 vec, softmax output)
        Returns:
            int: filtered prediction index
        """
        legal_tiles = np.where(input_tiles > 0)[1] 
        prediction = prediction.reshape(34,)
        filtered_prediction = np.zeros_like(prediction)
        filtered_prediction[legal_tiles] = prediction[legal_tiles]
        
        return np.argmax(filtered_prediction)

    def discard(self, hands_input):
        hands = np.zeros(34)

        for tile in hands_input:
            hands[tile['type'] * 9 + tile['index'] - 1] += 1

        hands = hands.reshape(1, 34)
        out = self.filter_prediction(hands, self.model.predict(hands))
        
        for i, tile in enumerate(hands_input):
            if tile['type'] * 9 + tile['index'] - 1 == out:
                return i